# Homework 2

In [1]:
import numpy as np
import pandas as pd
import html5lib

import requests
from bs4 import BeautifulSoup

## Paramethers for requests:
    1- ww_x_UNITE_ACAD - Section (Informatique?)  =  249847
    2- ww_x_PERIODE_ACAD - Semester wanted        =  978181
    3- ww_i_reportModel - General parameter =  133685247

In [2]:
unite_academique=249847
periode_academique=[978181, 978187,978195,39486325,123455150,213637754,213637922,213638028,355925344]
report_model=133685247



In [3]:
parameters={'ww_i_reportModel':'133685247','ww_i_reportModelXsl':'133685270','ww_x_UNITE_ACAD':'249847'}

In [4]:
base_all_gps='http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_b_list=1&ww_i_reportmodel=133685247&ww_c_langue=&ww_i_reportModelXsl=133685270&zz_x_UNITE_ACAD=Informatique'

In [5]:
informatique = {'ww_x_UNITE_ACAD': '249847'}
r = requests.get(base_all_gps,params=informatique)

In [6]:
r.text

'<html><head><META http-equiv="Content-Type" content="text/html; charset=iso-8859-1"><div></div><title></title><script type="text/javascript" src="GEDPUBLICREPORTS.txt?ww_x_path=Gestac.Base.Palette_js&amp;ww_c_langue=fr"></script><link rel="stylesheet" type="text/css" href="GEDPUBLICREPORTS.css?ww_x_path=Gestac.Moniteur.Style"><link rel="stylesheet" type="text/css" href="GEDPUBLICREPORTS.css?ww_x_path=Gestac.Moniteur.StyleNavigator"></head><body bgcolor="#ffffff" marginheight="0" marginwidth="5" link="#666666" vlink="#666666" alink="#666666"><div class="filtres"><form method="GET" name="f" action="!GEDPUBLICREPORTS.filter"><input type="hidden" name="ww_b_list" value="1"><input type="hidden" name="ww_i_reportmodel" value="133685247"><input type="hidden" name="ww_c_langue" value=""><h1 id="titre">Liste des étudiants inscrits par semestre</h1><table border="0" id="format"><tr><th>Format:</th></tr><tr><td><input type="radio" name="ww_i_reportModelXsl" value="133685270" checked>html</td></t

In [7]:
semester_bachelor=['Bachelor semestre 1','Bachelor semestre 2','Bachelor semestre 3','Bachelor semestre 4','Bachelor semestre 5','Bachelor semestre 6']
semester_master=['Master semestre 1','Master semestre 2','Master semestre 3','Projet Master automne','Projet Master printemps']
years=['2007-2008','2008-2009','2009-2010','2010-2011','2011-2012','2012-2013','2013-2014','2014-2015','2015-2016','2016-2017']

In [8]:
# Generate dictionary
main_dictionary={}
for year in years:
    main_dictionary[year]=dict()
    for item in semester_bachelor:
        main_dictionary[year][item]=0
    for item in semester_master:
        main_dictionary[year][item]=0
    

In [9]:
soup=BeautifulSoup(r.text, "lxml")

In [10]:
# Parse the parameter "periode_academique"

parse_years=soup.find_all('option')

for year in main_dictionary:
    for item in parse_years:
        if year==item.string:
            main_dictionary[year]['periode_academique']=str(item).split("\"")[1]

In [11]:
tag=soup.body

gps_list=tag.find_all('a')

In [12]:
# Process the GPS list
# Remove the ones without "Informatique", "Master" or "Bachelor"
# Hard preprocessing to fast eliminate the unwanted strings
gps_processed=[]
key1='Informatique'
key2='Bachelor'
key3='Master'
for i in gps_list:
    if key1 in i.contents[0] and (key2 in i.contents[0] or key3 in i.contents[0]):
        gps_processed.append(i)


In [13]:
# Keep only the right ones - get the GPS in the main_dictionary
for year in main_dictionary:
    for semester in main_dictionary[year]:
        for item in gps_processed:
            if 'Informatique, '+year+', '+semester==item.contents[0]:
                gps=str(item).split("'")[1].split("=")[1]
                main_dictionary[year][semester]=gps
print(main_dictionary)

{'2009-2010': {'Bachelor semestre 5': '71297605', 'Projet Master automne': '109592334', 'Bachelor semestre 4': '109576992', 'Bachelor semestre 6': '71297626', 'Bachelor semestre 1': '213617925', 'Bachelor semestre 2': '213617944', 'Projet Master printemps': '109592400', 'Bachelor semestre 3': '109576974', 'Master semestre 2': '213629869', 'Master semestre 1': '213629838', 'periode_academique': '978195', 'Master semestre 3': '109592375'}, '2007-2008': {'Bachelor semestre 5': '19561980', 'Projet Master automne': '39495382', 'Bachelor semestre 4': '39494823', 'Bachelor semestre 6': '19561998', 'Bachelor semestre 1': '71297531', 'Bachelor semestre 2': '71297550', 'Projet Master printemps': '39495444', 'Bachelor semestre 3': '39494788', 'Master semestre 2': '71450725', 'Master semestre 1': '71450699', 'periode_academique': '978181', 'Master semestre 3': '39495424'}, '2014-2015': {'Bachelor semestre 5': '1378362202', 'Projet Master automne': '1650773332', 'Bachelor semestre 4': '1650771947',

# Get the tables

In [14]:
base_request='http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html'

In [15]:
parameters={'ww_i_reportModel':'133685247','ww_i_reportModelXsl':'133685270'
            ,'ww_x_UNITE_ACAD':'249847','ww_x_PERIODE_ACAD':0,'ww_x_GPS':0}

In [16]:
bachelor_data={x:dict() for x in main_dictionary}
master_data={x:dict() for x in main_dictionary}
for year in main_dictionary:
    parameters['ww_x_PERIODE_ACAD']=main_dictionary[year]['periode_academique']
    for semester in main_dictionary[year]:
        if semester=='periode_academique':
            continue
        parameters['ww_x_GPS']=main_dictionary[year][semester]
        
        # Post the request
        req = requests.get(base_request,params=parameters)
        
        # Get the html file with BeautifulSoap (maybe not needed)
        #html_table=BeautifulSoup(req.text, "lxml")
        
        # Import and process the dataframe with pandas
        try:
            df=pd.read_html(str(req.text),skiprows=1,header=0)[0]
            df.columns=['Sex','Name','OrienB','OrienM','Specialization','Unuseful1','Mineur','State','Exchange','ExchangeSchool','Sciper','Unuseful2']
            del df['OrienB'],df['OrienM'],df['Unuseful1'],df['State'],df['Exchange'],df['ExchangeSchool'],df['Unuseful2']
            df['Year']=year
        except:
            df=pd.DataFrame({'Sex':[],'Name':[],'Specialization':[],'Mineur':[],'Sciper':[],'Year':[]})
        
        # Type semester can be 'Bachelor' or 'Master'. 
        type_semester=semester.split(' ')[0]
        if type_semester=='Projet':
            type_semester='Master'
            
        # Semester number is the number of the semester. It is 4 for master projects.
        semester_number=-1
        if semester!='Projet Master automne' and semester!='Projet Master printemps':
            semester_number=int(semester.split(' ')[2])
        else:
            semester_number=4
        df['Semester']=semester_number
        df=df.set_index(['Sciper','Year','Semester'])

        if type_semester=="Bachelor":
            bachelor_data[year][semester_number]=df
        elif type_semester=="Master":
            master_data[year][semester_number]=df
        else:
            print(type_semester)
            raise("Error parsing file: type semester not acceptable")
    print("Year "+year+" parsed")

Year 2009-2010 parsed
Year 2007-2008 parsed
Year 2014-2015 parsed
Year 2013-2014 parsed
Year 2016-2017 parsed
Year 2015-2016 parsed
Year 2011-2012 parsed
Year 2012-2013 parsed
Year 2010-2011 parsed
Year 2008-2009 parsed


In [17]:
# Check of correct parsing of all the semesters
bachelor_data_semesters=0
master_data_semesters=0
for year in master_data:
    master_data_semesters+=len(master_data[year])
for year in bachelor_data:
    bachelor_data_semesters+=len(bachelor_data[year])
if (bachelor_data_semesters+master_data_semesters!=100):
    raise("Some semester has not been correctly parsed")

# Bachelor analysis

In [18]:
bachelor_tables=[]
for year in bachelor_data:
    for semester in bachelor_data[year]:
        bachelor_tables.append(bachelor_data[year][semester])
total_bachelor=pd.concat(bachelor_tables,axis=0,join='outer')
total_bachelor=total_bachelor.reset_index()
groups_bachelor=total_bachelor.groupby('Sciper')

In [19]:
final_df_bachelor=pd.DataFrame({'Sciper':[],'Name':[],'Sex':[],'Semesters':[]})

for student,group in groups_bachelor:
    first_year=group['Semester'].isin([1]).sum()
    last_year=group['Semester'].isin([6]).sum()
    
    if first_year and last_year:
        number_of_years=group.shape[0]
        name=group['Name'].iloc[0]
        sex=group['Sex'].iloc[0]
        if sex=="Monsieur":
            sex='M'
        elif sex=="Madame":
            sex='F'
        else:
            raise("Not existing sex")
        ser=pd.DataFrame({'Sciper':[student],'Name':[name],'Sex':[sex],'Semesters':[number_of_years]})
        final_df_bachelor=final_df_bachelor.append(ser)
    
final_df_bachelor=final_df_bachelor.set_index('Sciper')
final_df_bachelor.head()

,Name,Semesters,Sex
Sciper,,,
147008.0,Good Xavier,6.0,M
169569.0,Arévalo Christian,6.0,M
169731.0,Knecht Mathieu,8.0,M
169795.0,Scheiben Pascal,9.0,M
171195.0,Richter Arnaud,6.0,M


In [20]:
final_df_bachelor[['Sex','Semesters']].groupby('Sex').describe()

Semesters
Sex                  
F   count   29.000000
    mean     6.793103
    std      1.346406
    min      6.000000
    25%      6.000000
    50%      6.000000
    75%      8.000000
    max     11.000000
M   count  368.000000
    mean     7.105978
    std      1.536891
    min      4.000000
    25%      6.000000
    50%      6.000000
    75%      8.000000
    max     12.000000

# Master analysis - TODO

In [21]:
master_tables=[]
for year in master_data:
    for semester in master_data[year]:
        master_tables.append(master_data[year][semester])
total_master=pd.concat(master_tables,axis=0,join='outer')
total_master=total_master.reset_index()
groups_master=total_master.groupby('Sciper')

In [ ]:
final_df_master=pd.DataFrame({'Sciper':[],'Name':[],'Sex':[],'Semesters':[],'Specialization':[]})

for student,group in groups_master:
    print(group)
    # Store the check variables
    first_semester=group['Semester'].isin([1]).sum()
    second_semester=group['Semester'].isin([2]).sum()
    third_semester=group['Semester'].isin([3]).sum()
    master_project_done=group['Semester'].isin([4]).sum()
    mineur=group['Mineur'].fillna('').sum()!=''
    specialization=group.Specialization.fillna(method='backfill').fillna(method='pad').fillna('').iloc[0]    
    year=group['Year'].max()

    # Store only the students with:
    #   - first_semester and master_project_done
    #   - first_semester and third_semester 
    #   - first_semester and second_semester and not mineur
    check1=first_semester and master_project_done
    check2=first_semester and third_semester
    check3=first_semester and second_semester and not mineur
    
    
    
    if check1 or check2 or check3:
        number_of_years=group.shape[0]
        name=group['Name'].iloc[0]
        sex=group['Sex'].iloc[0]
        if sex=="Monsieur":
            sex='M'
        elif sex=="Madame":
            sex='F'
        else:
            raise("Not existing sex")
        ser=pd.DataFrame({'Sciper':[student],'Name':[name],'Sex':[sex],'Semesters':[number_of_years],
                         'Specialization':[specialization],'Year':[year]})
        final_df_master=final_df_master.append(ser)
    
final_df_master=final_df_master.set_index('Sciper')
final_df_master.head()

       Sciper       Year  Semester Mineur            Name       Sex  \
260  128911.0  2007-2008         2    NaN  Gulati Asheesh  Monsieur   
336  128911.0  2007-2008         3    NaN  Gulati Asheesh  Monsieur   

         Specialization  
260  Internet computing  
336  Internet computing  
       Sciper       Year  Semester Mineur        Name       Sex Specialization
312  129093.0  2007-2008         2    NaN  Zhou Maoan  Monsieur            NaN
378  129093.0  2007-2008         3    NaN  Zhou Maoan  Monsieur            NaN
       Sciper       Year  Semester Mineur            Name       Sex  \
289  129326.0  2007-2008         2    NaN  Ni Zhong Zhong  Monsieur   
356  129326.0  2007-2008         3    NaN  Ni Zhong Zhong  Monsieur   

    Specialization  
289            NaN  
356            NaN  
       Sciper       Year  Semester Mineur                     Name       Sex  \
175  138088.0  2007-2008         1    NaN  Droz-dit-Busset Arnault  Monsieur   

    Specialization  
175         

In [23]:
final_df_master[['Sex','Semesters']].groupby('Sex').describe()

Semesters
Sex                  
F   count  101.000000
    mean     3.297030
    std      1.015328
    min      2.000000
    25%      3.000000
    50%      3.000000
    75%      4.000000
    max      7.000000
M   count  661.000000
    mean     3.284418
    std      0.972757
    min      2.000000
    25%      3.000000
    50%      3.000000
    75%      4.000000
    max      8.000000

In [26]:
specialization_analysis=final_df_master[['Semesters','Specialization']].groupby('Specialization')
specialization_analysis.count()

,Semesters
Specialization,
,519
Biocomputing,4
Computer Engineering - SP,20
Computer Science Theory,1
Data Analytics,6
Foundations of Software,62
Information Security - SP,7
Internet Information Systems,1
Internet computing,91


In [33]:
from scipy.stats import t

specialization_mean=specialization_analysis.mean()
specialization_std=specialization_analysis.std()
specialization_count=specialization_analysis.count()
output=specialization_analysis.mean()
base_mean=specialization_mean.iloc[0]
base_std=specialization_std.iloc[0]
base_count=specialization_count.iloc[0]
nu=specialization_count+base_count-2
prova=t(nu)
print(prova)
output['t-test']=(specialization_mean-base_mean)/(specialization_std*specialization_std/specialization_count+base_std*base_std/base_count)
#output['prova']=stats.t.sf(output['t-test'],nu)*2
output

,Semesters,t-test
Specialization,,
,3.055877,0.000000
Biocomputing,4.250000,2.117015
Computer Engineering - SP,3.600000,18.148938
Computer Science Theory,3.000000,NaN
Data Analytics,2.833333,-7.585569
Foundations of Software,3.919355,57.803129
Information Security - SP,3.285714,6.460912
Internet Information Systems,3.000000,NaN
Internet computing,3.824176,64.232246
